Pre-requisites: An empty wikibase. 
In this example a wikibase in wbstack.com is being used, but the script is working on the default Wikibase api so should work as well. 

These are the steps involved

##import the Wikidata integrator

In [1]:
%%capture
!pip install wikidataintegrator
from wikidataintegrator import wdi_core, wdi_login

## Get all Wikidata properties

In [2]:
query = """
SELECT DISTINCT ?property ?propertyLabel ?propertyDescription ?propType WHERE {
   ?property wikibase:directClaim ?p ;
             wikibase:propertyType ?propType ;
             schema:description ?propertyDescription ;
             rdfs:label ?propertyLabel .
   FILTER (lang(?propertyLabel) = 'en')
   FILTER (lang(?propertyDescription) = 'en')}
"""
propertiesSparql = wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe=True)
propertiesSparql
datatype_map = {'http://wikiba.se/ontology#CommonsMedia': 'commonsMedia',
                'http://wikiba.se/ontology#ExternalId': 'external-id',
                'http://wikiba.se/ontology#GeoShape': 'geo-shape',
                'http://wikiba.se/ontology#GlobeCoordinate': 'globe-coordinate',
                'http://wikiba.se/ontology#Math': 'math',
                'http://wikiba.se/ontology#Monolingualtext': 'monolingualtext',
                'http://wikiba.se/ontology#Quantity': 'quantity',
                'http://wikiba.se/ontology#String': 'string',
                'http://wikiba.se/ontology#TabularData': 'tabular-data',
                'http://wikiba.se/ontology#Time': 'time',
                'http://wikiba.se/ontology#Url': 'url',
                'http://wikiba.se/ontology#WikibaseItem': 'wikibase-item',
                'http://wikiba.se/ontology#WikibaseLexeme': 'lexeme',
                'http://wikiba.se/ontology#WikibaseForm': 'form',
                'http://wikiba.se/ontology#WikibaseSense': 'sense',
                'http://wikiba.se/ontology#MusicalNotation': 'musical-notation',
                'http://wikiba.se/ontology#WikibaseProperty': 'wikibase-property'}
propertiesSparql['datatype']= ""
for index, row in propertiesSparql.iterrows():
  row["datatype"] = datatype_map[row["propType"]] 
propertiesSparql

,property,propType,propertyLabel,propertyDescription,datatype
0,http://www.wikidata.org/entity/P10,http://wikiba.se/ontology#CommonsMedia,video,"relevant video. For images, use the property P...",commonsMedia
1,http://www.wikidata.org/entity/P610,http://wikiba.se/ontology#WikibaseItem,highest point,"point with highest elevation in a region, on a...",wikibase-item
2,http://www.wikidata.org/entity/P6,http://wikiba.se/ontology#WikibaseItem,head of government,"head of the executive power of this town, city...",wikibase-item
3,http://www.wikidata.org/entity/P181,http://wikiba.se/ontology#CommonsMedia,taxon range map image,range map of a taxon,commonsMedia
4,http://www.wikidata.org/entity/P180,http://wikiba.se/ontology#WikibaseItem,depicts,depicted entity (see also P921: main subject),wikibase-item
...,...,...,...,...,...
7987,http://www.wikidata.org/entity/P3368,http://wikiba.se/ontology#ExternalId,Prabook ID,identifier of a person in the Prabook database,external-id
7988,http://www.wikidata.org/entity/P3373,http://wikiba.se/ontology#WikibaseItem,sibling,the subject has the object as their sibling (b...,wikibase-item
7989,http://www.wikidata.org/entity/P3367,http://wikiba.se/ontology#ExternalId,GECD film ID,identifier of a film (movie) in the German Ear...,external-id
7990,http://www.wikidata.org/entity/P3366,http://wikiba.se/ontology#ExternalId,GECD person ID,identifier of a person in the German Early Cin...,external-id


In [3]:
propertiesSparql.columns

Index(['property', 'propType', 'propertyLabel', 'propertyDescription',
       'datatype'],
      dtype='object')

## Login to Wikibase

In [12]:
from getpass import getpass
import pprint

wikibase = "https://bh20subset1.wiki.opencura.com/"
api = wikibase+"w/api.php"
sparql = wikibase+"query/sparql"
entityUri = wikibase.replace("https:", "http:")+"entity/"
WBUSER = getpass(prompt="username:")  
WBPASS = getpass(prompt='Enter your password: ')  
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)

#### Function to create properties

In [13]:
## TODO: The mapping property in the target wikibase was made manually, but should
## be included in this script

## TODO 2: This script only considers English, but all languages supported should be added
def createProperty(login, wdprop, label, description, property_datatype):
  s = [wdi_core.WDUrl(wdprop, prop_nr="P1")]
  localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
  item = localEntityEngine(data=s)
  item.set_label(label)
  item.set_description(description)
  print(item.write(login, entity_type="property", property_datatype=property_datatype))
  

#### Create Wikidata properties

In [ ]:
datatype_map = {'http://wikiba.se/ontology#CommonsMedia': 'commonsMedia',
                'http://wikiba.se/ontology#ExternalId': 'external-id',
                'http://wikiba.se/ontology#GeoShape': 'geo-shape',
                'http://wikiba.se/ontology#GlobeCoordinate': 'globe-coordinate',
                'http://wikiba.se/ontology#Math': 'math',
                'http://wikiba.se/ontology#Monolingualtext': 'monolingualtext',
                'http://wikiba.se/ontology#Quantity': 'quantity',
                'http://wikiba.se/ontology#String': 'string',
                'http://wikiba.se/ontology#TabularData': 'tabular-data',
                'http://wikiba.se/ontology#Time': 'time',
                'http://wikiba.se/ontology#Url': 'url',
                'http://wikiba.se/ontology#WikibaseItem': 'wikibase-item',
                'http://wikiba.se/ontology#WikibaseProperty': 'wikibase-property'}

for row in propertiesSparql.itertuples():
  print(datatype_map[row.propType])
  print(row)
  createProperty(login, row.property,row.propertyLabel, row.propertyDescription, datatype_map[row.propType])

commonsMedia
Pandas(Index=0, property='http://www.wikidata.org/entity/P10', propType='http://wikiba.se/ontology#CommonsMedia', propertyLabel='video', propertyDescription='relevant video. For images, use the property P18. For film trailers, qualify with "object has role" (P3831)="trailer" (Q622550)', datatype='commonsMedia')


/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py:326: UserWarning: Warning: No distinct value properties found
Please set P2302 and Q21502410 in your wikibase or set `core_props` manually.
Continuing with no core_props
  "Continuing with no core_props")


P2
wikibase-item
Pandas(Index=1, property='http://www.wikidata.org/entity/P610', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='highest point', propertyDescription='point with highest elevation in a region, on a path, of a race', datatype='wikibase-item')
P3
wikibase-item
Pandas(Index=2, property='http://www.wikidata.org/entity/P6', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='head of government', propertyDescription='head of the executive power of this town, city, municipality, state, country, or other governmental body', datatype='wikibase-item')
P4
commonsMedia
Pandas(Index=3, property='http://www.wikidata.org/entity/P181', propType='http://wikiba.se/ontology#CommonsMedia', propertyLabel='taxon range map image', propertyDescription='range map of a taxon', datatype='commonsMedia')
P5
wikibase-item
Pandas(Index=4, property='http://www.wikidata.org/entity/P180', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='depicts', propertyD